<a href="https://colab.research.google.com/github/MRamiBalles/llm/blob/main/llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Librerías

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# Configuramos el dispositivo
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Estamos usando: {device}")

# Para reproducibilidad
torch.manual_seed(1337)

Estamos usando: cpu


2. Tokens

In [2]:
# Carga de datos
with open('datos_sancho_mini.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print("Longitud del dataset en caracteres: ", len(text))
print("\n--- Primeros 700 caracteres del dataset ---")
print(text[:700])

FileNotFoundError: [Errno 2] No such file or directory: 'datos_sancho_mini.txt'

Longitud del dataset en caracteres:  147758

--- Primeros 700 caracteres del dataset ---
<|inicio|>
Era de aspecto venerable y viejo;
de verde, azul y plata era el vestido,
robusto al parecer y de buen rejo,

aunque, como enojado, denegrido
se mostraba en el rostro, que la saña
así turba el color como el sentido.

Airado, contra aquéllos más se ensaña
que nadan más, y sáleles al paso,
juzgando a gloria tan cobarde hazaña.

En esto oh nuevo y milagroso caso,
digno de que se cuente poco a poco
y con los versos de Torcato Taso

Hasta aquí no he invocado, ahora invoco
vuestro favor, oh Musas, necesario
para los altos puntos en que toco;

descerrajad vuestro más rico almario,
y el aliento me dad que el caso pide,
no humilde, no ratero ni ordinario,
<|fin|>

<|inicio|>
que tiene a sus


In [ ]:
# Obtenemos todos los caracteres únicos que aparecen en el texto
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("--- Vocabulario ---")
print(''.join(chars))
print(f"Tamaño del vocabulario: {vocab_size}")

# Creamos el mapeo de caracteres a enteros (tokenización)
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # codificador: toma un string, devuelve una lista de enteros
decode = lambda l: ''.join([itos[i] for i in l]) # decodificador: toma una lista de enteros

# Probamos nuestra tokenización
test_string = "Hola Sancho"
encoded_string = encode(test_string)
decoded_string = decode(encoded_string)
print(f"\nPrueba de tokenización:")
print(f"{encoded_string}")

--- Vocabulario ---

 ,.;<>ABCDEFGHIJLMNOPQRSTUVXYZabcdefghijlmnopqrstuvxyz|ÁÉÍÑÓÚÜáéíñóúü
Tamaño del vocabulario: 70

Prueba de tokenización:
[5, 55, 39, 43, 39, 33, 39, 44, 55, 6, 0, 11, 47, 31, 1, 34, 35, 1, 31, 48, 45, 35, 33, 49, 44, 1, 51, 35, 43, 35, 47, 31, 32, 41, 35, 1, 53, 1, 51, 39, 35, 40, 44, 4, 0, 34, 35, 1, 51, 35, 47, 34, 35, 2, 1, 31, 54, 50, 41, 1, 53, 1, 45, 41, 31, 49, 31, 1, 35, 47, 31, 1, 35, 41, 1, 51, 35, 48, 49, 39, 34, 44, 2, 0, 47, 44, 32, 50, 48, 49, 44, 1, 31, 41, 1, 45, 31, 47, 35, 33, 35, 47, 1, 53, 1, 34, 35, 1, 32, 50, 35, 43, 1, 47, 35, 40, 44, 2, 0, 0, 31, 50, 43, 46, 50, 35, 2, 1, 33, 44, 42, 44, 1, 35, 43, 44, 40, 31, 34, 44, 2, 1, 34, 35, 43, 35, 37, 47, 39, 34, 44, 0, 48, 35, 1, 42, 44, 48, 49, 47, 31, 32, 31, 1, 35, 43, 1, 35, 41, 1, 47, 44, 48, 49, 47, 44, 2, 1, 46, 50, 35, 1, 41, 31, 1, 48, 31, 66, 31, 0, 31, 48, 65, 1, 49, 50, 47, 32, 31, 1, 35, 41, 1, 33, 44, 41, 44, 47, 1, 33, 44, 42, 44, 1, 35, 41, 1, 48, 35, 43, 49, 39, 34, 44, 3, 0, 0, 7

3. Embeddings

In [ ]:
n_embd = 256
block_size = 256

# Creamos las tablas de embeddings
token_embedding_table = nn.Embedding(vocab_size, n_embd)
position_embedding_table = nn.Embedding(block_size, n_embd)

# Ejemplo de cómo se combina:
# Tomemos un índice de token de ejemplo y una posición
idx_ejemplo = torch.tensor([[encode('Hola')[0]]], dtype=torch.long) # H -> 40
pos_ejemplo = torch.arange(0, 1, dtype=torch.long) # Posición 0

tok_emb = token_embedding_table(idx_ejemplo) # (1, 1, n_embd)
pos_emb = position_embedding_table(pos_ejemplo) # (1, n_embd)

x = tok_emb + pos_emb # Así se combinan

4. LayerNorm

In [ ]:
# PyTorch ya nos da una implementación eficiente
ln_ejemplo = nn.LayerNorm(n_embd)
# Aplicamos la normalización a nuestro embedding de ejemplo
x_normalizado = ln_ejemplo(x)

5. Self Attention

In [ ]:
n_head = 4
dropout = 0.2

class Head(nn.Module):
    """ Una cabeza de self-attention """
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * (C/4)**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    """ Múltiples cabezas de self-attention en paralelo """
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

6. MLP

In [ ]:
class FeedForward(nn.Module):
    """ Una red feed-forward simple """
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.GELU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

7. Bloque Transformer

In [ ]:
class Block(nn.Module):
    """ Un bloque de Transformer: comunicación seguida de computación """
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        # Primera parte: Self-Attention
        x = x + self.sa(self.ln1(x))
        # Segunda parte: Feed-Forward
        x = x + self.ffwd(self.ln2(x))
        return x

In [ ]:
n_layer = 4

class GPTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            # El softmax se aplica aquí para convertir logits en probabilidades
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

# Creamos la instancia del modelo y la movemos al dispositivo
model = GPTLanguageModel()

In [ ]:
8. Trainning

In [ ]:
# Primero, convertimos todo nuestro texto tokenizado en tensores de PyTorch
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

batch_size = 32

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

# Probamos nuestra función
xb, yb = get_batch('train')
print("--- Ejemplo de un lote de datos ---")
print('Entradas (shape):', xb.shape)
print('Targets (shape):', yb.shape)

--- Ejemplo de un lote de datos ---
Entradas (shape): torch.Size([32, 256])
Targets (shape): torch.Size([32, 256])


In [ ]:
max_iters = 5000
eval_interval = 200
learning_rate = 3e-4
eval_iters = 200

m = model.to(device)
print(f"Modelo creado con {sum(p.numel() for p in m.parameters())/1e6:.2f}M de parámetros.")
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# Bucle de entrenamiento
for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"Paso {iter}: pérdida de entrenamiento {losses['train']:.4f}, pérdida de validación {losses['val']:.4f}")

    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print("\n--- ¡Entrenamiento completado! ---")

Modelo creado con 3.26M de parámetros.
Paso 0: pérdida de entrenamiento 4.3902, pérdida de validación 4.3945
Paso 200: pérdida de entrenamiento 2.3039, pérdida de validación 2.3295
Paso 400: pérdida de entrenamiento 2.1847, pérdida de validación 2.2147
Paso 600: pérdida de entrenamiento 2.0520, pérdida de validación 2.0863
Paso 800: pérdida de entrenamiento 1.9414, pérdida de validación 1.9863
Paso 1000: pérdida de entrenamiento 1.8539, pérdida de validación 1.9126
Paso 1200: pérdida de entrenamiento 1.7761, pérdida de validación 1.8423
Paso 1400: pérdida de entrenamiento 1.7127, pérdida de validación 1.8018
Paso 1600: pérdida de entrenamiento 1.6375, pérdida de validación 1.7410
Paso 1800: pérdida de entrenamiento 1.5734, pérdida de validación 1.7012
Paso 2000: pérdida de entrenamiento 1.5168, pérdida de validación 1.6712
Paso 2200: pérdida de entrenamiento 1.4650, pérdida de validación 1.6374
Paso 2400: pérdida de entrenamiento 1.4166, pérdida de validación 1.6229
Paso 2600: pérdida 

9. Generación



In [ ]:
import time
import sys

print("\nsancho-mini generando texto :D")
print("\n")

# Cuanto más baja la temperatura más coherente, pero más predecible
temperature = 0.4
top_k = 50

start_token = encode('<|startofpoem|>')[0]
context = torch.tensor([[start_token]], dtype=torch.long, device=device)

initial_char = decode(context[0].tolist())
print(initial_char, end='')
sys.stdout.flush()

m.eval()
with torch.no_grad():
    for _ in range(1000):
        idx_cond = context[:, -block_size:]
        logits, _ = m(idx_cond)
        logits = logits[:, -1, :]
        logits = logits / temperature
        if top_k is not None:
            v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
            logits[logits < v[:, [-1]]] = -float('Inf') # Anula los logits que no están en el top-k

        # Aplicamos softmax a los logits ya modificados para obtener probabilidades
        probs = F.softmax(logits, dim=-1)
        idx_next = torch.multinomial(probs, num_samples=1)
        char = decode(idx_next[0].tolist())

        print(char, end='')
        sys.stdout.flush()
        context = torch.cat((context, idx_next), dim=1)
        time.sleep(0.02)




sancho-mini generando texto :D


<|inicio|>
de la espada en ella alta y la tierra
de la industria y entente brava
la fama que el aire de la cabeza lleva.

Mas no sé que estáis que es de aquí encierra
de la más nombre está ninguno
de aquellos tan tiene con la escucha,

de su guía el discreto se aprecia
el gran caso de la ocasión te ha vistoria.

En esto, que de los casos de los guía
el bajel que el cielo es de Apolo estaba,
y así le tierra en file corriente.

Cuatro vi tan al vez de la cabeza,
que sobre el cielo el cielo, y el cielo,
y en esto de la caballeza y crece,

por ver la vencidad y de la atrevida,
para en la canalla y la vitoria
de la caballeza y la fama por vida,
<|fin|>

<|inicio|>
de la caballera y la fama fuente,
y la muerte al cielo a la vitoria.

Mas no se le tierra en este repiese
a la canalla y con la vitoria y la fama
de la cabeza que el alma compaña;

de la caballera más de la cabeza
la vista, que el mundo el corte aprieto
de la insigne de la fama en poeta,
<|fin|>



10. Guardado final.

In [ ]:
!pip install huggingface_hub -q

# Iniciar sesión en tu cuenta de Hugging Face
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from huggingface_hub import HfApi, create_repo, upload_file
import torch

best_model_path = "sancho-mini-best.pth" # O el nombre que le hayas dado

MODEL_NAME = "sancho-mini.pth"
torch.save(m.state_dict(), MODEL_NAME)
print(f"Modelo guardado en '{MODEL_NAME}'")

# Creamos la tarjets de contenido para el modelo (README.md)

model_card_content = f"""
---
license: mit
language: es
pipeline_tag: text-generation
---

# sancho-mini: Un GPT a nivel de carácter que escribe sonetos como Cervantes

Este es un modelo Generative Pre-trained Transformer (GPT) entrenado desde cero con PyTorch para generar sonetos en español.

## Detalles del Modelo

- **Arquitectura**: GPT (Decoder-only Transformer)
- **Nivel de Tokenización**: Carácter
- **Tamaño del Vocabulario**: {vocab_size}
- **Dimensión de Embedding (`n_embd`)**: {n_embd}
- **Longitud de Contexto (`block_size`)**: {block_size}
- **Número de Capas Transformer (`n_layer`)**: {n_layer}
- **Número de Cabezas de Atención (`n_head`)**: {n_head}
- **Tasa de Dropout**: {dropout}
- **Número de Parámetros**: {sum(p.numel() for p in m.parameters())/1e6:.2f}M

## Datos de Entrenamiento

El modelo fue entrenado con un corpus de sonetos en español (`datos_sancho_mini.txt`), los poemas en el dataset están estructurados con tokens especiales para indicar el inicio y el fin de cada poema.

## Uso

Para usar este modelo, necesitas cargar el `state_dict` en una instancia de la arquitectura del modelo definida en el notebook de entrenamiento.
"""


try:
    with open("README.md", "w", encoding="utf-8") as f:
        f.write(model_card_content)
    print("Archivo 'README.md' creado correctamente.")
except Exception as e:
    print(f"Error al escribir README.md: {e}")


# Necesitamos el nombre de usuario

hf_username = input("Por favor introduce tu nombre de usuario de Hugging Face: ")
repo_name = "sancho-mini-gpt"
repo_id = f"{hf_username}/{repo_name}"

try:
    # Crear el repositorio
    repo_url = create_repo(repo_id, private=False, exist_ok=True) # Lo pongo público para que se vea fácil
    print(f"Repositorio creado (o ya existente): {repo_url}")

    # Subir el archivo del modelo
    upload_file(
        path_or_fileobj=MODEL_NAME,
        path_in_repo=MODEL_NAME,
        repo_id=repo_id,
        repo_type="model",
    )
    print(f"Archivo '{MODEL_NAME}' subido al repositorio.")

    # Subir la tarjeta de modelo
    upload_file(
        path_or_fileobj="README.md",
        path_in_repo="README.md",
        repo_id=repo_id,
        repo_type="model",
    )

    print(f"\nTu modelo está ahora en Hugging Face, puedes verlo aquí: {repo_url}")

except Exception as e:
    print(f"Problema: {e}")
    print("Asegúrate de haber introducido correctamente tu nombre de usuario y de tener un token con permisos de escritura.")

Para generación si el modelo ya está entrenado :

In [ ]:
model = GPTLanguageModel()
print(f"Modelo creado con {sum(p.numel() for p in model.parameters())/1e6:.2f}M de parámetros.")

MODEL_PATH = "sancho-mini.pth"

#    map_location=device asegura que el modelo se cargue en la GPU si está disponible.
print(f"Cargando pesos del modelo desde '{MODEL_PATH}'...")
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
print("¡Pesos cargados correctamente!")

m = model.to(device)

Modelo creado con 3.26M de parámetros.
Cargando pesos del modelo desde 'sancho-mini.pth'...
¡Pesos cargados correctamente!


In [ ]:
import time
import sys

print("\nsancho-mini generando texto :D")
print("\n")

# Parámetros de generación
temperature = 0.4
top_k = 50

# Ponemos el modelo en modo de evaluación
# Esto es importante porque desactiva capas como Dropout
m.eval()

# Token de inicio y contexto inicial
start_token = encode('<|startofpoem|>')[0]
context = torch.tensor([[start_token]], dtype=torch.long, device=device)

initial_char = decode(context[0].tolist())
print(initial_char, end='')
sys.stdout.flush()

# Generamos texto en un bloque no_grad para mayor eficiencia
with torch.no_grad():
    for _ in range(1000):
        idx_cond = context[:, -block_size:]
        logits, _ = m(idx_cond)
        logits = logits[:, -1, :]
        logits = logits / temperature

        if top_k is not None:
            v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
            logits[logits < v[:, [-1]]] = -float('Inf')

        probs = F.softmax(logits, dim=-1)
        idx_next = torch.multinomial(probs, num_samples=1)
        char = decode(idx_next[0].tolist())

        print(char, end='')
        sys.stdout.flush()
        context = torch.cat((context, idx_next), dim=1)
        time.sleep(0.02)
